# 텐서플로우 엔진을 기반한 딥러닝 개발방법

- 저수준 API로 개발
  - tensorflow api를 이용하여 개발
    - 1.x
    - 2.x
  - 내부를 직접 설계 한다
    - 장기 프로젝트 용도 적합
- 고수준 API로 개발
  - keras를 이용하여 개발
  - 제공된 틀에서만 개발 가능
    - 단기용, 장기도 필요하면 가능

In [2]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

TensorFlow 1.x selected.


'1.15.2'

In [3]:
# 코랩용 텐서보드 설치
!pip install tensorboardcolab

  Created wheel for tensorboardcolab: filename=tensorboardcolab-0.0.22-py3-none-any.whl size=3859 sha256=4451631b284f3ab1d53c8dab38c0125ed54a31ffea6e0e6ef87f07c863d08b0a
  Stored in directory: /root/.cache/pip/wheels/69/4e/4a/1c6c267395cb10edded1050df12af165d3254cfce324e80941
Successfully built tensorboardcolab


In [4]:
from tensorboardcolab import *
import os, shutil

Using TensorFlow backend.


In [5]:
# 텐서보드에 필요한 데이터 저장 공간 생성, 필요시 이전 기록 삭제
try:  
  if os.path.exists('./Graph'):                  # 존재하면
    shutil.rmtree('./Graph', ignore_errors=True) # 기존 자료 삭제
  # 폴더 생성
  os.mkdir('./Graph')
except Exception as e:
  print( e )

In [6]:
# 텐서보드 생성
tbc = TensorBoardColab()
# 오납용사례가 많아서 구글에서 블럭 요청 처리되어 이제 사용 불가 (아래링크)
# 코랩에 직집 빌트인해서 표현

Wait for 8 seconds...
TensorBoard link:
https://ea62-35-237-114-140.ngrok.io


In [7]:
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

# 데이터 획득
mnist     = input_data.read_data_sets('./data/mnist', one_hot=True)

# 환경변수
PIXEL     = mnist.train.images.shape[1]
PIXEL_H   = int( np.sqrt( PIXEL ) )
PIXEL_W   = PIXEL_H
LABEL_NUM = mnist.train.labels.shape[-1]

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [8]:
# y = xW + b
x = tf.placeholder( tf.float32, shape=( None, PIXEL ), name='x' )

def createFilterByWeight( name, shape ):
  name = f'{name}_W'
  initial_value = tf.truncated_normal( shape, stddev=0.1 )
  W = tf.Variable( initial_value=initial_value,  name=name )
  return W

def createBias( name, shape, value ):  
  name          = f'{name}_b'  
  initial_value = tf.constant( value, shape=shape )  
  b = tf.Variable( initial_value=initial_value, name=name)
  return b 

def createConv2D( name, x, W ):    
  name = f'{name}_conv'  
  return tf.nn.conv2d( x, filter=W, strides=[1,1,1,1], padding="SAME", name=name )

# 합성곱층 1층 생성
with tf.name_scope( '1f_conv' ) as scope:
  conv_1f_W   = createFilterByWeight( '1f_conv', (5, 5, 1, 32) )
  conv_1f_b   = createBias( '1f_conv', shape=(32,), value=0.1  )
  x_4d        = tf.reshape( x, ( -1, PIXEL_H, PIXEL_W, 1 ) )
  conv_1f     = createConv2D( '1f', x_4d, conv_1f_W ) + conv_1f_b
  act_conv_1f = tf.nn.relu( conv_1f )

def createMaxPooling( name, x ):  
  return tf.nn.max_pool( x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name=f'{name}_max')

# 풀링 1층 생성
with tf.name_scope( 'pooling_1f' ) as scope:  
  pool_1f = createMaxPooling( 'pooling_1f',  act_conv_1f)

# 합성곱 2층 생성
with tf.name_scope( '2f_conv' ) as scope:
  conv_2f_W   = createFilterByWeight( '2f_conv', (5, 5, 32, 32*2) )
  conv_2f_b   = createBias( '2f_conv', shape=(32*2,), value=0.1  )
  conv_2f     = createConv2D( '2f', pool_1f, conv_2f_W ) + conv_2f_b
  act_conv_2f = tf.nn.relu( conv_2f )

# 풀링 2층 생성
with tf.name_scope( 'pooling_2f' ) as scope:  
  pool_2f = createMaxPooling( 'pooling_2f',  act_conv_2f)

# 전결합층 생성 : Flattern
with tf.name_scope( 'fc' ) as scope:  
  _, h, w, ch  = pool_2f.shape
  in_channels  = h * w * ch # 784(원본픽셀) => 7*7*64 => 3306(합성곱2번통과후 픽셀수)
  out_channels = 1024 # 3306 -> 1024 -> 10
  fc_x   = tf.reshape( pool_2f, ( -1, in_channels ) ) # 4D -> 2D
  fc_W   = createFilterByWeight( 'fc', (in_channels, out_channels) ) # (?, 3306)*(3306, 1024)=>(?, 1024)
  fc_b   = createBias( 'fc', (out_channels, ), 0.1 )
  fc     = tf.matmul( fc_x , fc_W) + fc_b
  act_fc = tf.nn.relu( fc )

# 드롭아웃층 -> 과적합 방지를 위해서 학습을 방해 -> 신경망을 특정비율로 죽인다
with tf.name_scope( 'act_fc_dropout' ) as scope:
  keep_prob      = tf.placeholder( tf.float32 )
  act_fc_dropout = tf.nn.dropout( act_fc, rate=1-keep_prob)

# 출력층 생성
with tf.name_scope( 'output' ) as scope:
  _, in_ch = act_fc_dropout.shape
  y_W      = createFilterByWeight( 'ouptut', ( in_ch, LABEL_NUM ) ) # (?,1024)*(1024, 10) => (?,10)
  y_b      = createBias( 'ouptut', (LABEL_NUM, ), 0.1 )
  y_conv   = tf.matmul( act_fc_dropout, y_W ) + y_b
  y_conv   = tf.nn.softmax( y_conv )

# 실제 정답
y_ = tf.placeholder( tf.float32, shape=(None, LABEL_NUM), name='y_')

# 손실함수 
with tf.name_scope( 'loss' ) as scope:
  cross_entropy = -tf.reduce_sum( y_ * tf.log( y_conv ) )

# 최적화
with tf.name_scope( 'adam' ) as scope:
  optimizer = tf.train.AdamOptimizer()
  # 훈련도구 생성
  train     = optimizer.minimize( cross_entropy )

# 예측
with tf.name_scope( 'predict' ) as scope:
  predict   = tf.equal( tf.arg_max( y_conv, 1 ), tf.arg_max( y_, 1 ) )
  # 정확도
  accuracy  = tf.reduce_mean( tf.cast( predict, tf.float32 ) )

In [9]:
# 데이터 주입용 함수
def createFeedDict( x_train_data, y_train_label, prob ):
  return {
      x:x_train_data, 
      y_:y_train_label,
      keep_prob:prob
  }

# 학습
losses = list()
with tf.device('/device:GPU:0'):
  with tf.Session() as sess:  
    TRAIN_TOTAL_COUNT = 3000
    BATCH_SIZE        = 50  
    VERBOSE_INTERVAL  = 100     
    sess.run( tf.global_variables_initializer() )  
    test_fd = createFeedDict( mnist.test.images, mnist.test.labels, 1.0 )
    for step in range( TRAIN_TOTAL_COUNT ):    
      x_batch      = mnist.train.next_batch( BATCH_SIZE )    
      train_fd     = createFeedDict( x_batch[0], x_batch[1], 0.1 )   
      acc, _, loss = sess.run( [ accuracy, train, cross_entropy ], feed_dict=train_fd )
      #losses.append( loss )    
      if step % VERBOSE_INTERVAL == 0:    
        acc     = sess.run( accuracy, feed_dict=test_fd )
        print( f'step={step:4} acc={acc:20}, loss={loss:20}')
      pass    
    acc     = sess.run( accuracy, feed_dict=test_fd )
    print( f'step={step:4} acc={acc:20}, loss={loss:20}')

    # 텐서보드 데이터 기록
    writer = tbc.get_writer()      # 기록도구
    writer.add_graph( sess.graph ) # 기록
    writer.flush()                 # 강제 출력(강제로 기록되게 밀어낸다)

# 텐서보드 코랩 닫음
tbc.close()

step=   0 acc= 0.13740000128746033, loss=    1348.85595703125
step= 100 acc=  0.8801000118255615, loss=    53.7291259765625
step= 200 acc=  0.9365000128746033, loss=   33.26603698730469
step= 300 acc=  0.9538999795913696, loss=   20.76224136352539
step= 400 acc=  0.9591000080108643, loss=   20.31884002685547
step= 500 acc=  0.9624000191688538, loss=  14.143062591552734
step= 600 acc=  0.9667999744415283, loss=  15.246552467346191
step= 700 acc=  0.9696000218391418, loss=   8.383243560791016
step= 800 acc=  0.9718999862670898, loss=   8.990388870239258
step= 900 acc=  0.9746000170707703, loss=   7.577998161315918
step=1000 acc=   0.977400004863739, loss=  2.6912050247192383
step=1100 acc=  0.9764999747276306, loss=   2.016951322555542
step=1200 acc=  0.9783999919891357, loss=  15.434335708618164
step=1300 acc=  0.9807000160217285, loss=  6.4060869216918945
step=1400 acc=  0.9801999926567078, loss=   14.05860710144043
step=1500 acc=  0.9779000282287598, loss=  3.0947723388671875
step=160

In [10]:
%load_ext tensorboard

In [11]:
%tensorboard --logdir ./Graph